Experiment: Uses a weighted combination of al attention distribution
------
Summing together the last four attention distributions; using only one attention head.

In [3]:
import sys; sys.path.append('../..')
import torch
from bias_classification import prepare_model 
from src.utils import *

%load_ext autoreload
%autoreload 2

In [4]:
params = prepare_model.intialize_params("experiment_params.json")
dataset = prepare_model.initialize_dataset(params.intermediary_task)
attention_dataloader = dataset.return_dataloader()
attention_experiment = prepare_model.initialize_attention_experiment(params.intermediary_task)

06/04/2019 16:16:46 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
500it [00:00, 6277.88it/s]
06/04/2019 16:16:46 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
06/04/2019 16:16:46 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../../../tasks/bias_classification/results/cache/9c41111e2de8

In [5]:
# Run to print out the current key names in the dataset
dataset.get_key_names()

dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label'])

In [6]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

In [7]:
# Extracting indices of bias labels 
intermediary_labels = dataset.get_val('pre_tok_label_ids')
bias_indices = [label.flatten().tolist().index(1) for label in intermediary_labels]
# Extracting attention scores of biased token
biased_tok_dist = return_idx_attention_dist(attention_scores, bias_indices)
windowed_data, removed_indices = window_attention_dist(biased_tok_dist, bias_indices, window_size=params.final_task['window_size'])

In [8]:
processed_data = [torch.stack(list(attention_dict.values())) for attention_dict in windowed_data]

In [9]:
dataset.remove_indices(removed_indices)
dataset.add_data(torch.stack(processed_data), "attention_dist")
dataset.shuffle_data()

Run the cell below to reset params 
-----

In [10]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [11]:
print(params.final_task)

{'model': 'full_attentional', 'window_size': 5, 'num_attention_dists': 12, 'input_dim': 11, 'hidden_dim': 8, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.2, 'test_split': 0.1}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.001}}


Boostrapping procedure 

In [16]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [17]:
statistics

{'auc': [(0.5818637283768864, 0.8576365155312522), 0.7523553102563375],
 'accuracy': [(0.5346491228070175, 0.8070175438596491), 0.6778947368421053]}

New run 

In [18]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [19]:
print(params.final_task)

{'model': 'full_attentional', 'window_size': 5, 'num_attention_dists': 12, 'input_dim': 11, 'hidden_dim': 8, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.2, 'test_split': 0.1}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.001}}


Boostrapping procedure 

In [20]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [21]:
statistics

{'auc': [(0.574990633291717, 0.8700916479031804), 0.7566516868788632],
 'accuracy': [(0.5346491228070175, 0.8162280701754383), 0.6792982456140352]}

## No bootstrapping -- cannot calculate 95% confidence intervals 

In [173]:
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)
print(params.final_task)

{'model': 'full_attentional', 'window_size': 5, 'num_attention_dists': 12, 'input_dim': 11, 'hidden_dim': 8, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 3000, 'batch_size': 32, 'lr': 0.001}}


In [174]:
data_split = params.final_task['data_split']
batch_size = params.final_task['training_params']['batch_size']
train_dataloader, eval_dataloader, test_dataloader = dataset.split_train_eval_test(**data_split, batch_size=batch_size)

In [175]:
losses, evaluations = classification_experiment.train_model(train_dataloader, eval_dataloader, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [176]:
print(losses)

[[{'num_examples': 32, 'loss': 0.7852823734283447}, {'num_examples': 32, 'loss': 0.8070788979530334}, {'num_examples': 32, 'loss': 0.7824159860610962}, {'num_examples': 32, 'loss': 0.7458086013793945}, {'num_examples': 28, 'loss': 0.7125415205955505}], [{'num_examples': 32, 'loss': 0.7806649208068848}, {'num_examples': 32, 'loss': 0.8016020059585571}, {'num_examples': 32, 'loss': 0.7780117988586426}, {'num_examples': 32, 'loss': 0.742917001247406}, {'num_examples': 28, 'loss': 0.7112196087837219}], [{'num_examples': 32, 'loss': 0.7763853073120117}, {'num_examples': 32, 'loss': 0.796450674533844}, {'num_examples': 32, 'loss': 0.7738263607025146}, {'num_examples': 32, 'loss': 0.7401770949363708}, {'num_examples': 28, 'loss': 0.7099703550338745}], [{'num_examples': 32, 'loss': 0.7722666263580322}, {'num_examples': 32, 'loss': 0.7915153503417969}, {'num_examples': 32, 'loss': 0.7697829008102417}, {'num_examples': 32, 'loss': 0.7375678420066833}, {'num_examples': 28, 'loss': 0.7087981104850

[autoreload of bias_classification.prepare_model failed: Traceback (most recent call last):
  File "/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 376, in superreload
    module = reload(module)
  File "/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootst

In [177]:
avg_losses = [average_data(epoch_losses) for epoch_losses in losses]
avg_predictions = [average_data(epoch_evaluations) for epoch_evaluations in evaluations]

In [178]:
min_loss, max_loss, avg_loss = get_statistics(avg_losses, "loss")
print(min_loss)

0.35985812774071324


In [179]:
min_auc, max_auc, avg_auc = get_statistics(avg_predictions, "auc")

In [180]:
print(max_auc)

0.5817842288430524


In [181]:
print(min_auc)

0.3356251591545709
